In [ ]:
%reset -f
#
import time
import requests
import pandas as pd
from datetime import datetime
from pandas import json_normalize
#
pd.options.mode.chained_assignment = None
#
sensor_data = pd.read_csv('SensorData_Request.csv'); 
#
login_response = requests.request(
    "POST",
    "https://qsense.noolitic.com/api/v2/login",
    json={
        "username": "ChristosN", 
        "password": "qsenseair!"
        },
    headers={
        "content-type" : "application/json"
        }
    )
#
if login_response.status_code == 200:
    print('API log-in SUCCESS')
else:
    print('API log-in FAILED')
    print('Exiting...')
#
token = login_response.json()['token']
#
for i in range(0, len(sensor_data)):
    #
    start_date = sensor_data['InstallationDate'][i]
    start_date_unix = sensor_data['InstallationDateUNIX'][i]
    #
    end_date = pd.to_datetime(datetime.now())
    end_date_unix = int(time.mktime(end_date.timetuple())) 
    # end_date with pd_datetime timestamp
    
    #
    get_response = requests.request(
        "GET",
        "https://qsense.noolitic.com/api/v2/plugins/qsense/customer/datas/"+ sensor_data['SensorMAC'][i] + "?start=" + str(start_date_unix) + "&end=" + str(end_date_unix),
        json={
            "username": "ChristosN", 
            "password": "qsenseair!"
            },
        headers={
            "Content-type" : "application/json",
            "Authorization" : "Bearer " + token
            }
        )
    #
    if get_response.status_code == 200:
        #
        print('...')
        print('Parsing sensor '+str(sensor_data['SensorNo'][i]) + ' data: SUCCESS')
        print('...')
    else:
        print('Parsing sensor '+str(sensor_data['SensorNo'][i]) + ' data: FAILED')
    #
    timestamps = json_normalize(get_response.json().get('datas'))
    timestamps['timestamp'] = pd.to_datetime(timestamps['timeStamp'], unit='s'); 
    #
    response = json_normalize(timestamps['values'][0]); cols = list(response['name']); 
    data = pd.DataFrame(timestamps['timestamp']); data[cols]=""; 
    for j in range(0, len(timestamps)):
        for k in range(0, len(cols)):
            data[cols[k]][j] = timestamps['values'][j][k]['value']; 
    #
    data  = data.loc[data['timestamp'] > start_date].sort_values(by=['timestamp'], ascending=True).reset_index(drop=True); 
    #
    years = data['timestamp'][:].dt.year.unique()
    months = data['timestamp'][:].dt.month.unique()
    #
    data['year'] = data['timestamp'][:].dt.year
    data['month'] = data['timestamp'][:].dt.month
    data.rename(columns={'humidity': 'relative_humidity', 'ozone': 'o3'}, inplace=True)
    #
    print('Exporting data...')
    print('...')
    #
    # Rearrange columns
    if sensor_data['Anemometer'][i] == 'yes':
        data = data[['timestamp', 'wind_speed', 'wind_direction', 'temperature', 'pressure', 'relative_humidity', 'rain', 'no2', 'pm1', 'pm25', 'pm10', 'o3','year', 'month']]
    else:
        data = data[['timestamp', 'temperature', 'pressure', 'relative_humidity', 'no2', 'pm1', 'pm25', 'pm10', 'o3','year', 'month']]
    #
    # Replace NaN values with -9999
    data.fillna(-9999, inplace=True)
    #
    for m in range(0, len(years)):
        for n in range(0, len(months)):
            if len(data.loc[(data['year'] == years[m]) & (data['month'] == months[n])]) > 0:
                exec('S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + ' = data.loc[(data["year"] == years[m]) & (data["month"] == months[n])]')
                exec('S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.drop(columns=["year", "month"], inplace=True)')
                exec('S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.to_csv("./Data/S' + str(sensor_data['SensorNo'][i]) + '/S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.csv", index=False)')
                exec('del S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2))
                print('Exporting file S' + str(sensor_data['SensorNo'][i]) + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.csv : SUCCESS')
    #
    sensor_data['LastUpdate'][i] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sensor_data['LastUpdateUNIX'][i] = int(time.mktime(datetime.now().timetuple()))
    sensor_data['InstallationDateUNIX'][i] = int(time.mktime(pd.to_datetime(start_date).timetuple()))
#
print('...')
print('Updating sensor data...')
sensor_data.to_csv("SensorData_Request.csv", index=False)
print('Writing sensor data file: SUCCESS')
print('...')
#
del i, j, k, m, n, start_date, start_date_unix, end_date, end_date_unix, get_response, timestamps, response, cols, data, years, months, token, login_response
#
print('DONE')
